# NEUROIMAGING ANALYSIS AND RESEARCH (SUMMER 2025)

## GROUP: A

### Members:

**1. Furqaan Malik**

**2. Aryan Arya**

## Problem Statement

- Find early warning signs of brain disease by checking how brain networks break down over time in patients compared to healthy people.

## Setup and Imports

In [ ]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install nilearn
!pip install nibabel

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import nibabel as nib
import pickle
import collections
import statsmodels.api as sm
import seaborn as sns
import scipy.stats as stats

from sklearn.linear_model import LinearRegression
from nilearn import image, datasets, plotting
from nilearn.maskers import NiftiLabelsMasker
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from scipy.stats import ttest_ind, spearmanr
from statsmodels.stats.multitest import multipletests
from nilearn.plotting import plot_glass_brain

## 1. Data Organization

Since the scans are already denoised and in MNI space, here we:
- List and organize all scans by subject and session.
- Verify that spatial dimensions and affines are consistent across files.
- Perform a quick visual quality check.

### 1.1 Define data directory

In [ ]:
# Path to data directory
data_dir = '/content/drive/MyDrive/raw_data'

# Pattern matching all denoised MNI scans
file_pattern = os.path.join(data_dir, 'Denoised_*_MNI.nii.gz')
all_scans = sorted(glob.glob(file_pattern))

print(f"Found {len(all_scans)} scans")

Found 84 scans


### 1.2 Split control and patient files

In [ ]:
control_pattern = os.path.join(data_dir, 'Denoised_C*_MNI.nii.gz')
control_scans = sorted(glob.glob(control_pattern))

patient_pattern = os.path.join(data_dir, 'Denoised_p*_MNI.nii.gz')
patient_scans = sorted(glob.glob(patient_pattern))

print(f"Found {len(control_scans)} control scans and {len(patient_scans)} patient scans")

Found 36 control scans and 48 patient scans


##  2. ROI Definition & Time-Series Extraction

### 2.1  Load the Schaefer Atlas for ROI definition

We’ll use the Schaefer (2018) parcellation, selecting 200 regions at 2 mm resolution.

In [ ]:
# Fetch Schaefer 2018 atlas
atlas = datasets.fetch_atlas_schaefer_2018(n_rois=200, resolution_mm=2)
labels_img = atlas.maps
labels = atlas.labels
print(f"Loaded Schaefer atlas with {len(labels)} regions")

[fetch_atlas_schaefer_2018] Dataset found in /root/nilearn_data/schaefer_2018

Loaded Schaefer atlas with 201 regions


### 2.2 Initialize the Masker

Since we want to extract average Blood Oxygen Level Dependent (BOLD) signal from ROI, we will use `NiftiLabelsMasker` to extract the mean signal from each atlas region.

In [ ]:
masker = NiftiLabelsMasker(labels_img=labels_img,
                            standardize=False,
                            memory='nilearn_cache',
                            verbose=1)

### 2.3 Extract Time Series for Each Scan

In [ ]:
# We'll keep a dict mapping filename → 2D array (n_timepoints × n_regions)
time_series = {}

for fname in all_scans:
    ts = masker.fit_transform(fname)  # shape: (n_timepoints, 200)
    time_series[fname] = ts

print(f"Extracted time series for {len(time_series)} scans, each with shape {next(iter(time_series.values())).shape}")

[NiftiLabelsMasker.wrapped] loading data from 
/root/nilearn_data/schaefer_2018/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.nii.gz

________________________________________________________________________________
[Memory] Calling nilearn.maskers.base_masker.filter_and_extract...
filter_and_extract('/content/drive/MyDrive/raw_data/Denoised_C01_rs-1_MNI.nii.gz', <nilearn.maskers.nifti_labels_masker._ExtractionFunctor object at 0x7c5ddc4b4f10>, 
{ 'background_label': 0,
  'clean_args': None,
  'clean_kwargs': {},
  'cmap': 'CMRmap_r',
  'detrend': False,
  'dtype': None,
  'high_pass': None,
  'high_variance_confounds': False,
  'keep_masked_labels': True,
  'labels': None,
  'labels_img': '/root/nilearn_data/schaefer_2018/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.nii.gz',
  'low_pass': None,
  'lut': None,
  'mask_img': None,
  'reports': True,
  'smoothing_fwhm': None,
  'standardize': False,
  'standardize_confounds': True,
  'strategy': 'mean',
  't_r': None,
  'target_affine': None,
  'target_shape': None}, confounds=None, sample_mask=None, dtype=None, memory=Memory(location=nilearn_cache/joblib), mem

[NiftiLabelsMasker.wrapped] Loading data from /content/drive/MyDrive/raw_data/Denoised_C01_rs-1_MNI.nii.gz

[NiftiLabelsMasker.wrapped] Extracting region signals

[NiftiLabelsMasker.wrapped] Cleaning extracted signals

...

________________________________________________________________________________
[Memory] Calling nilearn.maskers.base_masker.filter_and_extract...
filter_and_extract('/content/drive/MyDrive/raw_data/Denoised_p16_rs-3_MNI.nii.gz', <nilearn.maskers.nifti_labels_masker._ExtractionFunctor object at 0x7c5d9c758150>, 
{ 'background_label': 0,
  'clean_args': None,
  'clean_kwargs': {},
  'cmap': 'CMRmap_r',
  'detrend': False,
  'dtype': None,
  'high_pass': None,
  'high_variance_confounds': False,
  'keep_masked_labels': True,
  'labels': None,
  'labels_img': '/root/nilearn_data/schaefer_2018/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.nii.gz',
  'low_pass': None,
  'lut': None,
  'mask_img': None,
  'reports': True,
  'smoothing_fwhm': None,
  'standardize': False,
  'standardize_confounds': True,
  'strategy': 'mean',
  't_r': None,
  'target_affine': None,
  'target_shape': None}, confounds=None, sample_mask=None, dtype=None, memory=Memory(location=nilearn_cache/joblib), mem

[NiftiLabelsMasker.wrapped] Loading data from /content/drive/MyDrive/raw_data/Denoised_p16_rs-3_MNI.nii.gz

[NiftiLabelsMasker.wrapped] Extracting region signals

[NiftiLabelsMasker.wrapped] Cleaning extracted signals

______________________________________________filter_and_extract - 28.1s, 0.5min
Extracted time series for 84 scans, each with shape (236, 200)


### 2.4 Build a df of timepoints and region signals for each scan

In [ ]:
# Build a list of records
records = []
for fname, ts in time_series.items():
    # ts is shape (T, 200)
    for t in range(ts.shape[0]):
        row = {'scan': os.path.basename(fname), 'timepoint': t}
        row.update({f"ROI_{i}": ts[t, i] for i in range(ts.shape[1])})
        records.append(row)

ts_df = pd.DataFrame.from_records(records)
ts_df.head()

,scan,timepoint,ROI_0,ROI_1,ROI_2,ROI_3,ROI_4,ROI_5,ROI_6,ROI_7,...,ROI_190,ROI_191,ROI_192,ROI_193,ROI_194,ROI_195,ROI_196,ROI_197,ROI_198,ROI_199
0,Denoised_C01_rs-1_MNI.nii.gz,0,2.325405,-0.906794,3.296173,-3.729747,1.765465,1.859265,-1.530835,3.340798,...,-4.165909,-5.306962,-2.128399,-3.132540,-4.423469,-3.031343,-2.115925,0.761606,-1.236421,-0.829004
1,Denoised_C01_rs-1_MNI.nii.gz,1,1.655761,-1.505065,2.337543,-3.663717,0.885289,1.934811,-2.094643,3.284882,...,-2.068610,-4.106806,-2.544708,-2.766587,-4.570967,-2.313492,-1.761983,1.013404,-0.787222,-0.695933
2,Denoised_C01_rs-1_MNI.nii.gz,2,0.864392,-2.215235,1.104975,-3.770922,-0.204032,1.773100,-2.826010,2.964158,...,-0.231913,-2.813040,-2.612766,-2.474446,-4.617666,-1.668673,-1.431533,1.125409,-0.301132,-0.441941
3,Denoised_C01_rs-1_MNI.nii.gz,3,-0.006823,-2.916015,-0.278159,-3.987339,-1.385188,1.322681,-3.616983,2.352622,...,0.726197,-1.862982,-2.306731,-2.387442,-4.522084,-1.207866,-1.211636,1.055626,0.199821,-0.038357
4,Denoised_C01_rs-1_MNI.nii.gz,4,-0.866180,-3.436343,-1.599590,-4.172763,-2.485925,0.593504,-4.300085,1.475205,...,0.267268,-1.691268,-1.673991,-2.598500,-4.244554,-1.022724,-1.190191,0.786042,0.677211,0.498154


### 2.5 Save the extracted time series

In [ ]:
output_path = os.path.join(data_dir, 'time_series_dict.pkl')
with open(output_path, 'wb') as f:
    pickle.dump(time_series, f)

print(f"Saved time series dict to {output_path}")

Saved time series dict to /content/drive/MyDrive/raw_data/time_series_dict.pkl


## 3. Compute per‐region variability

In [ ]:
# Load the time series file
output_path = os.path.join(data_dir, 'time_series_dict.pkl')
with open(output_path, 'rb') as f:
    time_series = pickle.load(f)

### 3.1 Compute variability (std) in each region across all time-points for each scan

In [ ]:
# Compute std for each region across time for each scan
variability = {
    fname: ts.std(axis=0) for fname, ts in time_series.items()
}

# Build a DataFrame
roi_cols = [f"ROI_{i}" for i in range(next(iter(variability.values())).shape[0])]
var_df = pd.DataFrame.from_dict(variability, orient='index', columns=roi_cols)
var_df.index.name = 'scan'
var_df.reset_index(inplace=True)

var_df.head()

,scan,ROI_0,ROI_1,ROI_2,ROI_3,ROI_4,ROI_5,ROI_6,ROI_7,ROI_8,...,ROI_190,ROI_191,ROI_192,ROI_193,ROI_194,ROI_195,ROI_196,ROI_197,ROI_198,ROI_199
0,/content/drive/MyDrive/raw_data/Denoised_C01_r...,1.605341,2.337575,2.131480,2.926725,3.277017,2.801390,4.830689,2.283908,2.220401,...,12.094370,9.932542,3.677664,5.393105,3.181712,4.753881,3.810434,2.910377,2.353492,2.417642
1,/content/drive/MyDrive/raw_data/Denoised_C01_r...,1.114963,2.935813,1.705017,2.469147,4.871023,2.099069,5.755569,1.980255,4.228475,...,9.340275,6.943423,2.950129,3.739832,2.070822,2.976509,2.755897,3.049572,2.842664,3.735765
2,/content/drive/MyDrive/raw_data/Denoised_C01_r...,2.927762,7.177607,5.862166,6.060543,10.293820,3.827666,12.910129,4.026211,9.292055,...,1.202379,5.315594,5.153642,3.515115,2.825149,5.639696,4.712777,4.828084,5.206172,6.612858
3,/content/drive/MyDrive/raw_data/Denoised_C02_r...,2.517980,3.923062,2.677515,4.195957,4.344526,1.496369,6.454253,2.290159,5.227879,...,3.167315,3.384230,2.205581,2.704285,2.440419,3.884986,2.935590,2.827439,3.638058,3.292448
4,/content/drive/MyDrive/raw_data/Denoised_C02_r...,1.365584,2.535571,2.617238,3.157567,3.028446,1.692062,4.546976,3.259797,4.168501,...,3.457377,3.670710,2.098629,4.417566,3.235526,3.723499,2.227788,2.669691,3.366681,2.887252


### 3.2 Extract subject and session from file name and group controls & patients

In [ ]:
# Extract subject & session
for path in all_scans:
  fname = os.path.basename(path)
  parts = fname.split('_')
  subject = parts[1]                      # C01, p04, etc
  session = int(parts[2].split('-')[1])   # 1, 2, 3

  var_df.loc[var_df['scan'] == path, 'subject'] = subject
  var_df.loc[var_df['scan'] == path, 'session'] = session

# Group controls & patients
var_df['group'] = var_df['subject'].str.startswith('C').map({True: 'control', False: 'patient'})

var_df.head()

,scan,ROI_0,ROI_1,ROI_2,ROI_3,ROI_4,ROI_5,ROI_6,ROI_7,ROI_8,...,ROI_193,ROI_194,ROI_195,ROI_196,ROI_197,ROI_198,ROI_199,subject,session,group
0,/content/drive/MyDrive/raw_data/Denoised_C01_r...,1.605341,2.337575,2.131480,2.926725,3.277017,2.801390,4.830689,2.283908,2.220401,...,5.393105,3.181712,4.753881,3.810434,2.910377,2.353492,2.417642,C01,1.0,control
1,/content/drive/MyDrive/raw_data/Denoised_C01_r...,1.114963,2.935813,1.705017,2.469147,4.871023,2.099069,5.755569,1.980255,4.228475,...,3.739832,2.070822,2.976509,2.755897,3.049572,2.842664,3.735765,C01,2.0,control
2,/content/drive/MyDrive/raw_data/Denoised_C01_r...,2.927762,7.177607,5.862166,6.060543,10.293820,3.827666,12.910129,4.026211,9.292055,...,3.515115,2.825149,5.639696,4.712777,4.828084,5.206172,6.612858,C01,3.0,control
3,/content/drive/MyDrive/raw_data/Denoised_C02_r...,2.517980,3.923062,2.677515,4.195957,4.344526,1.496369,6.454253,2.290159,5.227879,...,2.704285,2.440419,3.884986,2.935590,2.827439,3.638058,3.292448,C02,1.0,control
4,/content/drive/MyDrive/raw_data/Denoised_C02_r...,1.365584,2.535571,2.617238,3.157567,3.028446,1.692062,4.546976,3.259797,4.168501,...,4.417566,3.235526,3.723499,2.227788,2.669691,3.366681,2.887252,C02,2.0,control


## 4. (Optional) Build control baselines & compute patient z-scores

Now that we have each scan’s per-ROI variability in var_df, we’ll:

1. Compute, for each ROI, the mean and std of variability across all control scans.

2. Merge those baselines back onto var_df and, for patient rows, compute

$$ z-score\ =\ \frac{x - μ}{σ}  $$

- This is done to get an idea of which regions defer the most in controls vs patients. Hence, Setp 4 is **optional**.


### 4.1 Compute control baselines

In [ ]:
roi_cols = [c for c in var_df.columns if c.startswith('ROI_')]
ctrl_df = var_df[var_df['group']=='control']

baselines = pd.DataFrame({
    'ROI': roi_cols,
    'ctrl_mean': ctrl_df[roi_cols].mean().values,
    'ctrl_std': ctrl_df[roi_cols].std().values,
})

# turn into dicts for fast lookup
mean_map = dict(zip(baselines['ROI'], baselines['ctrl_mean']))
std_map  = dict(zip(baselines['ROI'], baselines['ctrl_std']))

### 4.2 Compute pateints' z-scores

In [ ]:
patient_df = var_df[var_df['group']=='patient'].copy()

# Prepare your ROI list and baseline Series
roi_cols   = [c for c in var_df.columns if c.startswith('ROI_')]
ctrl_means = pd.Series(mean_map)    # index = 'ROI_i', values = control means
ctrl_stds  = pd.Series(std_map)     # index = 'ROI_i', values = control stds

# z-score computation
z_df = (
    patient_df[roi_cols]
    .subtract(ctrl_means, axis=1)        # patient_df[roi] - control_mean
    .div(ctrl_stds + 1e-8, axis=1)       # / control_std
)
z_df.columns = [c + '_z' for c in roi_cols]

# Concatenate back onto patient_df in one shot
patient_df = pd.concat([patient_df, z_df], axis=1)

patient_df.head()

,scan,ROI_0,ROI_1,ROI_2,ROI_3,ROI_4,ROI_5,ROI_6,ROI_7,ROI_8,...,ROI_190_z,ROI_191_z,ROI_192_z,ROI_193_z,ROI_194_z,ROI_195_z,ROI_196_z,ROI_197_z,ROI_198_z,ROI_199_z
36,/content/drive/MyDrive/raw_data/Denoised_p01_r...,1.736980,2.038797,3.944206,4.888185,3.235929,1.875313,5.215375,4.085690,4.122474,...,0.964123,-0.074216,0.221423,0.276284,-0.143680,-1.408812,-1.424676,-1.373870,-1.223126,-1.200645
37,/content/drive/MyDrive/raw_data/Denoised_p01_r...,2.765586,3.405374,5.569689,5.645340,6.533579,2.431163,6.924487,3.400078,6.646292,...,0.730898,-0.148858,1.162186,0.075603,1.126510,-0.817822,-0.807914,-1.273032,-0.695317,-0.899810
38,/content/drive/MyDrive/raw_data/Denoised_p01_r...,1.588066,2.874317,3.354217,4.379899,4.714576,1.627656,5.441312,2.386513,4.479458,...,-0.139571,-0.006018,0.528701,1.788954,1.282824,-0.041068,-0.241495,-0.595457,-0.754558,-0.079277
39,/content/drive/MyDrive/raw_data/Denoised_p02_r...,4.206203,5.359783,4.768195,6.174993,6.845204,4.606702,10.388179,5.339890,7.960947,...,-0.816372,-0.862645,-0.816860,0.348535,0.093813,-1.489968,-0.643656,1.204029,1.035759,1.696917
40,/content/drive/MyDrive/raw_data/Denoised_p02_r...,2.918385,2.803646,3.673354,3.455336,2.340362,5.351758,4.357622,6.301980,3.024425,...,-0.312408,-0.305330,-0.621993,-0.418744,-0.305195,-1.503885,-1.251459,0.523459,1.003500,1.144453


### 4.3 Analyze regions that are 2 std away from control baseline

In [ ]:
# thresholds
z_thresh = 2.0

# dirs
results_dir = os.path.join(data_dir, "results")
plots_dir = os.path.join(results_dir, "plots")

os.makedirs(results_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)

for idx, row in patient_df.iterrows():
    fname    = row['scan']
    subj, sess = row['subject'], row['session']

    # collect the z-values
    z_vals = np.array([row[f"ROI_{i}_z"] for i in range(len(roi_cols))])

    # zero out everything below the threshold
    outlier_vals = np.zeros_like(z_vals)
    mask = np.abs(z_vals) > z_thresh
    outlier_vals[mask] = z_vals[mask]

    # turn into a brain image
    z_map = masker.inverse_transform(outlier_vals.reshape(1, -1))

    # quick printout of which labels are outliers
    out_idx = np.where(mask)[0]
    print(f"{subj} session {sess}: {len(out_idx)} outlier regions")
    for i in out_idx:
        print(f"  • {labels[i]}: z={z_vals[i]:.2f}")

    # plot and save a glass brain view
    display = plotting.plot_glass_brain(
        z_map, threshold=z_thresh, plot_abs=False,
        title=f"{subj} S{sess} |z|>{z_thresh}"
    )
    png_path = os.path.join(plots_dir, f"{subj}_s{sess}_zglass.png")
    display.savefig(png_path)
    display.close()


[NiftiLabelsMasker.inverse_transform] computing image from signals

p01 session 1.0: 5 outlier regions
  • 7Networks_LH_SalVentAttn_FrOperIns_2: z=2.06
  • 7Networks_LH_Default_Temp_1: z=2.04
  • 7Networks_RH_DorsAttn_Post_2: z=2.33
  • 7Networks_RH_Limbic_OFC_1: z=2.09
  • 7Networks_RH_Cont_PFCl_6: z=-2.18


[NiftiLabelsMasker.inverse_transform] computing image from signals

p01 session 2.0: 2 outlier regions
  • 7Networks_RH_SalVentAttn_Med_3: z=2.49
  • 7Networks_RH_Limbic_OFC_3: z=2.05


[NiftiLabelsMasker.inverse_transform] computing image from signals

...

p16 session 1.0: 32 outlier regions
  • 7Networks_LH_Vis_14: z=-2.15
  • 7Networks_LH_SomMot_1: z=-2.32
  • 7Networks_LH_SomMot_4: z=-2.31
  • 7Networks_LH_SomMot_6: z=-2.45
  • 7Networks_LH_DorsAttn_Post_2: z=-2.16
  • 7Networks_LH_DorsAttn_Post_5: z=-2.28
  • 7Networks_LH_DorsAttn_Post_10: z=-2.25
  • 7Networks_LH_DorsAttn_FEF_1: z=-2.40
  • 7Networks_LH_DorsAttn_FEF_2: z=-2.01
  • 7Networks_LH_SalVentAttn_ParOper_1: z=-2.60
  • 7Networks_LH_SalVentAttn_ParOper_2: z=-2.19
  • 7Networks_LH_SalVentAttn_FrOperIns_3: z=-2.01
  • 7Networks_LH_SalVentAttn_Med_2: z=-2.03
  • 7Networks_LH_Cont_Par_1: z=-2.49
  • 7Networks_LH_Cont_OFC_1: z=-2.30
  • 7Networks_LH_Cont_PFCl_2: z=-2.65
  • 7Networks_LH_Cont_PFCl_3: z=-2.15
  • 7Networks_LH_Default_Par_3: z=-2.23
  • 7Networks_LH_Default_PFC_2: z=-2.08
  • 7Networks_RH_SomMot_7: z=-2.16
  • 7Networks_RH_SomMot_9: z=-2.68
  • 7Networks_RH_SomMot_10: z=-2.06
  • 7Networks_RH_DorsAttn_Post_6: z=-2.04
  • 7Networks_RH_DorsAttn_Post_10: z=-2.50
  • 7N

[NiftiLabelsMasker.inverse_transform] computing image from signals

p16 session 2.0: 3 outlier regions
  • 7Networks_LH_Cont_Par_1: z=-2.18
  • 7Networks_LH_Default_Par_3: z=-2.12
  • 7Networks_RH_DorsAttn_Post_10: z=-2.23


[NiftiLabelsMasker.inverse_transform] computing image from signals

p16 session 3.0: 2 outlier regions
  • 7Networks_LH_SalVentAttn_ParOper_1: z=-2.14
  • 7Networks_RH_Cont_PFCl_6: z=-2.35


## 5. Build connectivity matrices and graphs

In [ ]:
# 5a) Parameters
threshold_prop = 0.2   # keep top 20% strongest connections

# 5b) Containers
conn_matrices = {}
graphs        = {}

for fname, ts in time_series.items():
    # 1) Correlation matrix
    corr = np.corrcoef(ts.T)            # shape: (200,200)
    np.fill_diagonal(corr, 0)           # zero out self‐connections

    # 2) Proportional threshold
    #    find the cutoff so that only the top `threshold_prop` fraction remains
    cutoff = np.percentile(np.abs(corr), (1 - threshold_prop) * 100)
    thr_corr = corr.copy()
    thr_corr[np.abs(thr_corr) < cutoff] = 0

    adj = np.abs(thr_corr)

    # 3) Build graph
    G = nx.from_numpy_array(adj)   # weighted graph by default

    # 4) Store
    conn_matrices[fname] = thr_corr
    graphs[fname]        = G

print(f"Built graphs for {len(graphs)} scans")


Built graphs for 84 scans


## 6. Compute network metrics

- Now we already have region wise variability. Alongside, we will also analyze how brain regions intereact as a network. Hence, we will compute some global metrics that summarize overall network integrity.

In [ ]:
from networkx.algorithms import efficiency_measures as em

metrics = {}

for fname, G in graphs.items():
    # 6a) Ensure connected subgraph for path length
    if not nx.is_connected(G):
        # take the largest connected component
        comp = max(nx.connected_components(G), key=len)
        Gconn = G.subgraph(comp).copy()
    else:
        Gconn = G

    # 6b) Compute metrics
    metrics[fname] = {
        'clustering':       nx.average_clustering(G, weight='weight'),
        'path_length':      nx.average_shortest_path_length(Gconn, weight='weight'),
        'global_eff':       em.global_efficiency(G),
        'local_eff':        em.local_efficiency(G),
        'mean_strength':    np.mean([d for _, d in G.degree(weight='weight')]),
        'mean_betweenness': np.mean(list(nx.betweenness_centrality(G, weight='weight').values()))
    }

# 6c) Turn into DataFrame
metrics_df = pd.DataFrame.from_dict(metrics, orient='index').reset_index().rename(columns={'index':'scan'})

In [ ]:
# Extract subject & session
for path in all_scans:
  fname = os.path.basename(path)
  parts = fname.split('_')
  subject = parts[1]                      # C01, p04, etc
  session = int(parts[2].split('-')[1])   # 1, 2, 3

  metrics_df.loc[metrics_df['scan'] == path, 'subject'] = subject
  metrics_df.loc[metrics_df['scan'] == path, 'session'] = session

# Group controls & patients
metrics_df['group'] = metrics_df['subject'].str.startswith('C').map({True: 'control', False: 'patient'})

metrics_df.head()

,scan,clustering,path_length,global_eff,local_eff,mean_strength,mean_betweenness,subject,session,group
0,/content/drive/MyDrive/raw_data/Denoised_C01_r...,0.438736,1.316314,0.550188,0.787736,28.095240,0.005605,C01,1.0,control
1,/content/drive/MyDrive/raw_data/Denoised_C01_r...,0.309552,0.982905,0.582936,0.739589,23.891927,0.004572,C01,2.0,control
2,/content/drive/MyDrive/raw_data/Denoised_C01_r...,0.352087,1.094947,0.567994,0.748556,25.111561,0.005031,C01,3.0,control
3,/content/drive/MyDrive/raw_data/Denoised_C02_r...,0.356852,1.119027,0.558671,0.750239,24.950939,0.005342,C02,1.0,control
4,/content/drive/MyDrive/raw_data/Denoised_C02_r...,0.270062,0.874173,0.587178,0.716339,22.148269,0.004247,C02,2.0,control


## 7. Fit slopes accross time for each feature

- We have 200 features from `var_df` and 6 features from `metrics_df`.
- We will now compute slopes of these 206 features over the 3 annual scans to get a trend of how brain network changes over time.

In [ ]:
# 1) Variability slopes
roi_cols = [c for c in var_df.columns if c.startswith('ROI_')]
# melt to long form
vars_long = var_df.melt(['subject','session'], roi_cols,
                        var_name='feature', value_name='value')

# compute slope per (subject, feature)
var_slopes = (
    vars_long
      .groupby(['subject','feature'])
      .apply(lambda df: np.polyfit(df['session'], df['value'], 1)[0])
      .reset_index(name='slope')
)
# pivot to wide: one row per subject
var_slopes_wide = var_slopes.pivot(index='subject', columns='feature', values='slope')
var_slopes_wide.reset_index(inplace=True)

# 2) Network‐metric slopes
metric_cols = ['clustering','path_length','global_eff','local_eff',
               'mean_strength','mean_betweenness']
mets_long = metrics_df.melt(['subject','session'], metric_cols,
                            var_name='feature', value_name='value')

met_slopes = (
    mets_long
      .groupby(['subject','feature'])
      .apply(lambda df: np.polyfit(df['session'], df['value'], 1)[0])
      .reset_index(name='slope')
)
met_slopes_wide = met_slopes.pivot(index='subject', columns='feature', values='slope')
met_slopes_wide.reset_index(inplace=True)

# 3) Merge variability‐slopes & metric‐slopes
slopes_df = (
    var_slopes_wide
      .merge(met_slopes_wide, on='subject', suffixes=('_var','_net'))
)

# 4) Add group label
group_map = var_df.set_index('subject')['group'].to_dict()
slopes_df['group'] = slopes_df['subject'].map(group_map)

slopes_df.head()

/tmp/ipython-input-32-440934362.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: np.polyfit(df['session'], df['value'], 1)[0])
/tmp/ipython-input-32-440934362.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: np.polyfit(df['session'], df['value'], 1)[0])


feature,subject,ROI_0,ROI_1,ROI_10,ROI_100,ROI_101,ROI_102,ROI_103,ROI_104,ROI_105,...,ROI_97,ROI_98,ROI_99,clustering,global_eff,local_eff,mean_betweenness,mean_strength,path_length,group
0,C01,0.661210,2.420016,1.424748,0.307021,0.321776,2.454869,2.395391,2.679783,3.940825,...,0.873657,1.155210,0.632369,-0.043325,0.008903,-0.019590,-0.000287,-1.491839,-0.110683,control
1,C02,0.141125,0.246956,1.317506,0.028524,-0.109327,-0.152613,-0.477515,0.563680,0.325825,...,0.843138,0.347248,0.578128,-0.021707,0.010840,-0.009976,-0.000347,-0.770736,-0.073671,control
2,C04,0.475268,0.753913,1.185675,0.043395,0.074121,0.515106,1.103081,0.484538,1.200132,...,0.455446,1.290259,-0.001149,-0.006405,-0.000278,-0.007896,0.000008,0.069920,0.005026,control
3,C05,0.578308,0.272481,-0.129932,-0.084891,0.297721,0.444903,1.139369,0.229468,0.886491,...,-0.494802,1.555789,-0.049278,-0.021497,0.003522,-0.008366,-0.000099,-1.162120,-0.070187,control
4,C06,0.002249,0.103865,0.033579,-0.054239,0.113649,0.490234,0.127075,0.314508,1.093894,...,-0.104569,0.655559,-0.034324,0.001225,0.003256,0.005266,-0.000099,-0.188169,-0.020630,control


## 8. Find and validate biomarkers usig Stratified K Fold cross validation and Statistical testing (t-test)

In [ ]:
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
import pandas as pd

# 8a) Identify the slope columns
feat_cols = [c for c in slopes_df.columns if c not in ['subject','group']]
X = slopes_df[feat_cols].values
y = slopes_df['group'].map({'control':0,'patient':1}).values
subjects = slopes_df['subject'].values

### Set up Stratified K folds

In [ ]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# To store per‐fold performance
fold_metrics = []

# To count how often each feature is selected across folds
feature_counts = collections.Counter()

In [ ]:
for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), start=1):
    # Split to train/test DataFrames
    train_df = slopes_df.iloc[train_idx].reset_index(drop=True)
    test_df  = slopes_df.iloc[test_idx].reset_index(drop=True)

    # --- 4a) Feature selection on TRAIN only ---
    #  run t-tests feature-by-feature
    p_vals = []
    for feat in feat_cols:
        ctrl = train_df.loc[train_df.group=='control', feat]
        pat  = train_df.loc[train_df.group=='patient', feat]
        _, p = ttest_ind(pat, ctrl, nan_policy='omit')
        p_vals.append(p)
    # FDR correction
    reject, q_vals = multipletests(p_vals, alpha=0.05, method='fdr_bh')[:2]
    selected = [f for f, r in zip(feat_cols, reject) if r]
    # if none survive, pick top-10 by raw p
    if not selected:
        selected = [f for f,_ in sorted(zip(feat_cols, p_vals), key=lambda x: x[1])][:10]

    # record selection frequency
    for f in selected:
        feature_counts[f] += 1

    # --- 4b) Train a regularized classifier ---
    X_tr = train_df[selected].values
    y_tr = train_df['group'].map({'control':0,'patient':1}).values
    X_te = test_df[selected].values
    y_te = test_df['group'].map({'control':0,'patient':1}).values

    clf = LogisticRegression(penalty='l1', solver='saga', max_iter=5000)
    clf.fit(X_tr, y_tr)

    # --- 4c) Evaluate on TEST ---
    y_pred = clf.predict(X_te)
    y_prob = clf.predict_proba(X_te)[:,1]

    acc = accuracy_score(y_te, y_pred)
    auc = roc_auc_score(y_te, y_prob)

    fold_metrics.append({
        'fold': fold,
        'n_train': len(train_idx),
        'n_test':  len(test_idx),
        'n_feats': len(selected),
        'accuracy': acc,
        'auc':      auc
    })

    print(f"Fold {fold}: selected {len(selected)} features, "
          f"acc={acc:.2f}, AUC={auc:.2f}")


Fold 1: selected 3 features, acc=0.83, AUC=0.78
Fold 2: selected 10 features, acc=0.83, AUC=0.89
Fold 3: selected 2 features, acc=0.83, AUC=0.88
Fold 4: selected 3 features, acc=0.80, AUC=1.00
Fold 5: selected 1 features, acc=0.80, AUC=1.00


## 9. Report & visualize the significant biomarkers

In [ ]:
# 5a) CV performance
perf_df = pd.DataFrame(fold_metrics)
print("\nCross-validation summary:")
print(perf_df[['fold','n_feats','accuracy','auc']])
print("Mean ± SD accuracy:",
      f"{perf_df.accuracy.mean():.2f} ± {perf_df.accuracy.std():.2f}")
print("Mean ± SD AUC:",
      f"{perf_df.auc.mean():.2f} ± {perf_df.auc.std():.2f}")



Cross-validation summary:
   fold  n_feats  accuracy       auc
0     1        3  0.833333  0.777778
1     2       10  0.833333  0.888889
2     3        2  0.833333  0.875000
3     4        3  0.800000  1.000000
4     5        1  0.800000  1.000000
Mean ± SD accuracy: 0.82 ± 0.02
Mean ± SD AUC: 0.91 ± 0.09


In [ ]:
# 5b) Feature selection frequency
freq_df = pd.DataFrame.from_records(
    list(feature_counts.items()),
    columns=['feature','count']
).sort_values('count', ascending=False).reset_index(drop=True)
print("\nTop features by selection frequency across folds:")
print(freq_df.head(5))


Top features by selection frequency across folds:
   feature  count
0   ROI_98      5
1   ROI_70      4
2  ROI_108      3
3  ROI_103      1
4  ROI_109      1


In [ ]:
img_dir = os.path.join(results_dir, "images")
os.makedirs(img_dir, exist_ok=True)

def plot_glass_rois(rois, masker, region_labels, threshold=0.5, output_dir=img_dir):

    os.makedirs(output_dir, exist_ok=True)

    # Ensure we have exactly 200 labels to match the masker
    # Some atlas.label lists include a “background” at index 0; drop it if present.
    if len(region_labels) == len(masker.labels_):
        region_labels = region_labels[1:]
    n_rois = len(region_labels)

    for idx in rois:
        # Build a one‐hot array of length n_rois
        arr = np.zeros((1, n_rois))
        arr[0, idx] = 1

        # Inverse‐transform back to a NIfTI
        img = masker.inverse_transform(arr)

        # Plot
        display = plotting.plot_glass_brain(
            img,
            threshold=threshold,
            colorbar=False,
            title=f"ROI {idx}: {region_labels[idx]}"
        )

        # Save out
        fname = os.path.join(output_dir, f"ROI_{idx}_glass.png")
        display.savefig(fname)
        display.close()
        print(f" → Saved ROI {idx} ({region_labels[idx]}) to {fname}")


# Example usage for your top 3 biomarkers:
plot_glass_rois([98, 70, 108], masker, labels, threshold=0.5)

[NiftiLabelsMasker.inverse_transform] computing image from signals

 → Saved ROI 98 (7Networks_LH_Default_pCunPCC_4) to /content/drive/MyDrive/raw_data/results/images/ROI_98_glass.png


[NiftiLabelsMasker.inverse_transform] computing image from signals

 → Saved ROI 70 (7Networks_LH_Cont_pCun_1) to /content/drive/MyDrive/raw_data/results/images/ROI_70_glass.png


[NiftiLabelsMasker.inverse_transform] computing image from signals

 → Saved ROI 108 (7Networks_RH_Vis_9) to /content/drive/MyDrive/raw_data/results/images/ROI_108_glass.png


## 10. Validate classifiers accuracy using only top 3 regions

In [ ]:
# 1) Pick your three biomarker features
features = ['ROI_98', 'ROI_70', 'ROI_108']

# 2) Build X and y
X = slopes_df[features].values
y = slopes_df['group'].map({'control':0,'patient':1}).values

# 3) Define the classifier
clf = LogisticRegression(penalty='l1', solver='saga', max_iter=5000)

# 4) Set up stratified 5-fold CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 5) Compute accuracy and AUC
acc_scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
auc_scores = cross_val_score(clf, X, y, cv=cv, scoring='roc_auc')

# 6) Print results
print(f"Accuracy: {acc_scores.mean():.2f} ± {acc_scores.std():.2f}")
print(f"AUC:      {auc_scores.mean():.2f} ± {auc_scores.std():.2f}")


Accuracy: 0.89 ± 0.09
AUC:      0.96 ± 0.09


## 11. Analyze correlations between top 3 regions and clinical scores

In [ ]:
# 1) Load clinical CSV
clin_df = pd.read_csv('/content/drive/MyDrive/raw_data/files/demoSCA7.csv')

# 2) Compute per-subject slopes for clinical measures
measures = ['moca', 'mmse', 'sara']
clin_slopes = []
for subj, group in clin_df.groupby('subject'):
    rec = {'subject': subj}
    sessions = group['session'].values
    for m in measures:
        values = group[m].values
        # fit a line: slope = coefficient of session
        if np.isnan(values).any():
            rec[m + '_slope'] = np.nan
        else:
            rec[m + '_slope'] = np.polyfit(sessions, values, 1)[0]
    clin_slopes.append(rec)
clin_slopes_df = pd.DataFrame(clin_slopes)

clin_slopes_df.head()

,subject,moca_slope,mmse_slope,sara_slope
0,p01,1.0,-1.500000e+00,0.75
1,p02,-2.0,-1.000000e+00,3.75
2,p03,0.5,-2.121350e-15,2.00
3,p04,-1.5,-1.000000e+00,2.25
4,p05,-2.5,-4.263770e-15,2.50


In [ ]:
# 3) Prepare ROI slopes for top 3 biomarkers
roi_slopes_df = (
    slopes_df[['subject', 'ROI_98', 'ROI_70', 'ROI_108']]
    .rename(columns={'ROI_98': 'roi98_slope',
                     'ROI_70': 'roi70_slope',
                     'ROI_108': 'roi108_slope'})
)

# 4) Merge clinical slopes with ROI slopes
merge_df = pd.merge(roi_slopes_df, clin_slopes_df, on='subject')

merge_df.head()

,subject,roi98_slope,roi70_slope,roi108_slope,moca_slope,mmse_slope,sara_slope
0,p01,0.260756,-0.527109,-0.906743,1.0,-1.500000e+00,0.75
1,p02,-1.066329,-0.850625,-2.053650,-2.0,-1.000000e+00,3.75
2,p03,-1.066880,-0.774394,-0.390960,0.5,-2.121350e-15,2.00
3,p04,-1.286051,-0.900751,-0.304527,-1.5,-1.000000e+00,2.25
4,p05,-0.298578,-0.343579,0.707736,-2.5,-4.263770e-15,2.50


In [ ]:
# 5) Compute Spearman correlations
results = []
for roi in ['roi98_slope', 'roi70_slope', 'roi108_slope']:
    for m in measures:
        corr, pval = spearmanr(merge_df[roi], merge_df[m + '_slope'], nan_policy='omit')
        results.append({
            'roi': roi,
            'clinical_measure': m,
            'spearman_rho': corr,
            'p_value': pval
        })
results_df = pd.DataFrame(results)

# 6) Display results
print("Correlation between ROI slopes and clinical score slopes:")
print(results_df.to_string(index=False))

Correlation between ROI slopes and clinical score slopes:
         roi clinical_measure  spearman_rho  p_value
 roi98_slope             moca     -0.022075 0.935325
 roi98_slope             mmse     -0.373158 0.154565
 roi98_slope             sara     -0.514706 0.041346
 roi70_slope             moca     -0.313466 0.237108
 roi70_slope             mmse     -0.070797 0.794446
 roi70_slope             sara     -0.476471 0.062059
roi108_slope             moca     -0.572480 0.020474
roi108_slope             mmse     -0.063422 0.815495
roi108_slope             sara     -0.261765 0.327410
